# Camera calibration

## Setup for intrinsic calibration

In [ ]:
import cv2
import numpy as np

# Define the chessboard size
chessboard_size = (8, 8)

# Prepare object points, like (0,0,0), (1,0,0), ..., (8,5,0)
object_points = np.zeros((chessboard_size[0] * chessboard_size[1], 3), np.float32)
object_points[:, :2] = np.mgrid[0:chessboard_size[0], 0:chessboard_size[1]].T.reshape(-1, 2)

# Arrays to store object points and image points from all the images
object_points_list = []  # 3D points in real world space
image_points_list = []  # 2D points in image plane

# Load the images
image_paths = ['image1.jpg', 'image2.jpg', 'image3.jpg']  # We need 10 images
for image_path in image_paths:
    # Read the image
    image = cv2.imread(image_path)
    
    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, chessboard_size, None)
    
    # If corners are found, add object points and image points
    if ret == True:
        object_points_list.append(object_points)

        # cv.cornerSubPix is used to refine the corner locations
        corners2 = cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria=(cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001))
        image_points_list.append(corners2)
        
        # Draw and display the corners
        cv2.drawChessboardCorners(image, chessboard_size, corners2, ret)
        cv2.imshow('Chessboard Corners', image)
        cv2.waitKey(0)




### Calibration

In [ ]:
# Calibrate the camera,  function returns the camera matrix, distortion coefficients, rotation and translation vectors etc.
ret, camera_matrix, distortion_coeffs, rvecs, tvecs = cv2.calibrateCamera(
    object_points_list, image_points_list, gray.shape[::-1], None, None)

# Print the camera matrix and distortion coefficients
print("Camera Matrix:")
print(camera_matrix)
print("\nDistortion Coefficients:")
print(distortion_coeffs)

### Undistortion

In [ ]:
import cv2
import numpy as np

# Load the distorted image and refine camera matrix
distorted_image = cv2.imread('distorted_image.jpg')
h, w = distorted_image.shape[:2]
newcamera_matrix, roi = cv2.getOptimalNewCameraMatrix(camera_matrix, distortion_coeffs, (w, h), 1, (w, h))



# Undistort the image
undistorted_image = cv2.undistort(distorted_image, camera_matrix, distortion_coeffs, None, newcamera_matrix)

# Crop the image
x, y, w, h = roi
undistorted_image = undistorted_image[y:y+h, x:x+w]
cv2.imwrite('calibresult.png', undistorted_image)

# Display the undistorted image
cv2.imshow('Undistorted Image', undistorted_image)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [ ]:
# Store matrix and distortion coefficients
np.savez('calibration_data.npz', camera_matrix=camera_matrix, distortion_coeffs=distortion_coeffs)

### Re-projection error

In [ ]:
mean_error = 0
for i in range(len(object_points_list)):
    image_points, _ = cv2.projectPoints(object_points_list[i], rvecs[i], tvecs[i], camera_matrix, distortion_coeffs)
    error = cv2.norm(image_points_list[i], image_points, cv2.NORM_L2) / len(image_points)
    mean_error += error

print("Total error: ", mean_error / len(object_points_list))

## Extrinsic calibration

In [ ]:
import cv2
import numpy as np

# Define the object points in 3D space
object_points = np.array([[0, 0, 0], [1, 0, 0], [0, 1, 0], [1, 1, 0]], dtype=np.float32)

# Define the corresponding image points in 2D space
image_points = np.array([[10, 20], [30, 40], [50, 60], [70, 80]], dtype=np.float32)

# Load camera matrix and distortion coefficients
calibration_data = np.load('calibration_data.npz')
camera_matrix = calibration_data['camera_matrix']
distortion_coeffs = calibration_data['distortion_coeffs']

# Find the extrinsic parameters
retval, rvec, tvec = cv2.solvePnP(object_points, image_points, camera_matrix, distortion_coeffs)

# Print the rotation vector and translation vector
print("Rotation Vector:")
print(rvec)
print("\nTranslation Vector:")
print(tvec)


In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Define the rotation vector and translation vector
rvec = [...]  # Replace [...] with your actual rotation vector
tvec = [...]  # Replace [...] with your actual translation vector

# Create a 3D plot
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Plot the camera position
ax.scatter(0, 0, 0, c='r', marker='o')  # Assuming the camera is at the origin

# Plot the object points
object_points = [...]  # Replace [...] with your actual object points
ax.scatter(object_points[:, 0], object_points[:, 1], object_points[:, 2], c='b', marker='o')

# Plot the camera frame
camera_frame_length = 0.1  # Length of the camera frame axes
camera_frame_endpoints = [
    (0, 0, 0),
    (camera_frame_length, 0, 0),
    (0, camera_frame_length, 0),
    (0, 0, camera_frame_length)
]
for i, j in [(0, 1), (0, 2), (0, 3)]:
    ax.plot([camera_frame_endpoints[i][0], camera_frame_endpoints[j][0]],
            [camera_frame_endpoints[i][1], camera_frame_endpoints[j][1]],
            [camera_frame_endpoints[i][2], camera_frame_endpoints[j][2]], c='r')

# Apply the rotation and translation to the object points
rotated_object_points = [...]  # Replace [...] with your actual rotated object points

# Plot the rotated object points
ax.scatter(rotated_object_points[:, 0], rotated_object_points[:, 1], rotated_object_points[:, 2], c='g', marker='o')

# Set the plot limits and labels
ax.set_xlim([-1, 1])
ax.set_ylim([-1, 1])
ax.set_zlim([-1, 1])
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')

# Show the plot
plt.show()


In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Create a 3D plot
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Plot your data points and customize the plot

# Save the plot as a PNG image file
plt.savefig('plot.png')
